In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.feature_selection import RFE
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score, confusion_matrix
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier


Using matplotlib backend: TkAgg


In [3]:
# importation de donnees 
df = pd.read_csv("database_TT_Churn.csv")
df 

df.head()

,id_client,genre,age,marie,num_tel,nb_jours_abonne,duree_appel_jour,nb_appel_jour,cout_appel_jour,duree_appel_soiree,...,nb_appel_nuit,cout_appel_nuit,duree_appel_inter,nb_appel_inter,cout_appel_inter,active_msg_vocaux,nb_msg_vocaux,nb_reclamation,churn,offer_type
0,382-4657,femme,37,yes,98505453,128.0,265.1,110.0,45.07,197.4,...,91.0,11.01,10.0,3.0,2.70,yes,25,1.0,False,Hayya
1,371-7191,homme,46,no,97321658,107.0,161.6,123.0,27.47,195.5,...,103.0,11.45,13.7,3.0,3.70,yes,26,1.0,False,PRE - 1=11
2,358-1921,homme,50,no,98653270,137.0,243.4,114.0,41.38,121.2,...,104.0,7.32,12.2,5.0,3.29,no,0,0.0,False,PRE - 900 bonus
3,375-9999,homme,78,yes,96303256,84.0,299.4,71.0,50.90,61.9,...,89.0,8.86,6.6,7.0,1.78,no,0,2.0,False,PRE - AHLA
4,330-6626,femme,75,yes,96412387,75.0,166.7,113.0,28.34,148.3,...,121.0,8.41,10.1,3.0,2.73,no,0,3.0,False,PRE - Binetna


In [4]:
df.shape

(5000, 23)

In [5]:
df.isna().sum()

id_client              0
genre                  0
age                    0
marie                  0
num_tel                0
nb_jours_abonne       10
duree_appel_jour       0
nb_appel_jour          7
cout_appel_jour        0
duree_appel_soiree     0
nb_appel_soiree        6
cout_appel_soiree      0
duree_appel_nuit       0
nb_appel_nuit         11
cout_appel_nuit        0
duree_appel_inter      0
nb_appel_inter        12
cout_appel_inter       0
active_msg_vocaux      8
nb_msg_vocaux          0
nb_reclamation        12
churn                  4
offer_type             0
dtype: int64

In [6]:
# information sur l'ensemble de donnees
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id_client           5000 non-null   object 
 1   genre               5000 non-null   object 
 2   age                 5000 non-null   int64  
 3   marie               5000 non-null   object 
 4   num_tel             5000 non-null   int64  
 5   nb_jours_abonne     4990 non-null   float64
 6   duree_appel_jour    5000 non-null   float64
 7   nb_appel_jour       4993 non-null   float64
 8   cout_appel_jour     5000 non-null   float64
 9   duree_appel_soiree  5000 non-null   float64
 10  nb_appel_soiree     4994 non-null   float64
 11  cout_appel_soiree   5000 non-null   float64
 12  duree_appel_nuit    5000 non-null   float64
 13  nb_appel_nuit       4989 non-null   float64
 14  cout_appel_nuit     5000 non-null   float64
 15  duree_appel_inter   5000 non-null   float64
 16  nb_app

In [7]:
df.describe().T


,count,mean,std,min,25%,50%,75%,max
age,5000.0,4.650680e+01,16.743513,19.0,3.200000e+01,46.00,60.00,80.00
num_tel,5000.0,9.844445e+07,371630.462063,90032100.0,9.850545e+07,98505453.00,98505453.00,99652014.00
nb_jours_abonne,4990.0,1.002707e+02,39.704130,1.0,7.300000e+01,100.00,127.00,243.00
duree_appel_jour,5000.0,1.802889e+02,53.894699,0.0,1.437000e+02,180.10,216.20,351.50
nb_appel_jour,4993.0,1.000190e+02,19.828023,0.0,8.700000e+01,100.00,113.00,165.00
cout_appel_jour,5000.0,3.064967e+01,9.162069,0.0,2.443000e+01,30.62,36.75,59.76
duree_appel_soiree,5000.0,2.006366e+02,50.551309,0.0,1.663750e+02,201.00,234.10,363.70
nb_appel_soiree,4994.0,1.002147e+02,19.820534,0.0,8.700000e+01,100.00,114.00,170.00
cout_appel_soiree,5000.0,1.705432e+01,4.296843,0.0,1.414000e+01,17.09,19.90,30.91
duree_appel_nuit,5000.0,2.003916e+02,50.527789,0.0,1.669000e+02,200.40,234.70,395.00


In [8]:
# nombre de valeurs uniques par colonne 
df.nunique()

id_client             5000
genre                    2
age                     62
marie                    2
num_tel                 33
nb_jours_abonne        218
duree_appel_jour      1961
nb_appel_jour          123
cout_appel_jour       1961
duree_appel_soiree    1879
nb_appel_soiree        126
cout_appel_soiree     1659
duree_appel_nuit      1853
nb_appel_nuit          131
cout_appel_nuit       1028
duree_appel_inter      170
nb_appel_inter          21
cout_appel_inter       170
active_msg_vocaux        2
nb_msg_vocaux           48
nb_reclamation          10
churn                    2
offer_type              40
dtype: int64

In [9]:
#ANALYSE UNIVARIEE

In [10]:
# varaiables catregorielles
categorical_columns = ['genre', 'marie', 'offer_type','active_msg_vocaux', 'churn']
categorical_columns

['genre', 'marie', 'offer_type', 'active_msg_vocaux', 'churn']

In [11]:
# creation d une fonction de construction de histogrammes a barres et de maniere interactive
def plot_distribution(column):
    
    if column == 'churn':
        # Plot donut chart for 'churn' column
        plt.figure(figsize=(8, 6))
        df[column].value_counts().plot(kind='pie', autopct='%1.1f%%', colors=['#ff9999','#66b3ff'], startangle=90, wedgeprops=dict(width=0.3), labels=None)
        centre_circle = plt.Circle((0,0),0.70,fc='white')
        fig = plt.gcf()
        fig.gca().add_artist(centre_circle)
        plt.axis('equal')  
        plt.title('Churn Distribution')
        plt.legend(labels=df[column].value_counts().index, loc='best')
        plt.show()
    else:
        # Plot bar chart for other columns
        plt.figure(figsize=(8, 6))
        ax = sns.countplot(x=column, data=df, palette='coolwarm')
        plt.ylabel('Nombre de Clients')
        plt.title('Distribution de ' + column)
        plt.xticks(rotation=45)
        
        # Add percentage labels to the bars for columns other than 'offer_type'
        if column != 'offer_type':
            total = len(df[column])
            for p in ax.patches:
                percentage = '{:.1f}%'.format(100 * p.get_height() / total)
                x = p.get_x() + p.get_width() / 2 - 0.05
                y = p.get_height()
                ax.annotate(percentage, (x, y), ha='center', va='bottom')
        
        plt.show()

# Now you can use interact
interact(plot_distribution, column=categorical_columns)


interactive(children=(Dropdown(description='column', options=('genre', 'marie', 'offer_type', 'active_msg_voca…

<function __main__.plot_distribution(column)>

In [12]:
# varaiables quantitatives
numerical_columns = ['age','nb_jours_abonne','duree_appel_jour','nb_appel_jour','cout_appel_jour','duree_appel_soiree','nb_appel_soiree',
                    'cout_appel_soiree','duree_appel_nuit','nb_appel_nuit','cout_appel_nuit','duree_appel_inter','nb_appel_inter',
                    'cout_appel_inter','nb_msg_vocaux','nb_reclamation']
numerical_columns

['age',
 'nb_jours_abonne',
 'duree_appel_jour',
 'nb_appel_jour',
 'cout_appel_jour',
 'duree_appel_soiree',
 'nb_appel_soiree',
 'cout_appel_soiree',
 'duree_appel_nuit',
 'nb_appel_nuit',
 'cout_appel_nuit',
 'duree_appel_inter',
 'nb_appel_inter',
 'cout_appel_inter',
 'nb_msg_vocaux',
 'nb_reclamation']

In [13]:
# Distribution d'age
age_intervals = [(0, 18), (18, 30), (30, 50), (50, 70), (70, 120)]

# Create a new column in the DataFrame to represent age intervals
df['age_interval'] = pd.cut(df['age'], bins=[interval[0] for interval in age_intervals] + [np.inf], labels=[f'{interval[0]}-{interval[1]}' for interval in age_intervals])

# Plot the bar chart
plt.figure(figsize=(10, 6))
sns.countplot(x='age_interval', data=df,  color='#FFB6C1',order=[f'{interval[0]}-{interval[1]}' for interval in age_intervals])
plt.title("Distribution d'âge")
plt.xlabel('Age')
plt.ylabel('Nombre de clients')
plt.legend( loc='upper right')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


In [14]:
#Distribution de Nombre d'Appel pour chaque type d'appel




# Total number of calls for each type of call
appel_jour = df['nb_appel_jour'].sum()
appel_soiree = df['nb_appel_soiree'].sum()
appel_nuit = df['nb_appel_nuit'].sum()
appel_inter = df['duree_appel_inter'].sum()

# Labels for each type of call
labels = ['Appels de Jour', 'Appels de Soirée', 'Appels de Nuit', 'Appels Internationaux']

x = np.arange(len(labels))
width = 0.35

# Creating the bar plot
fig, ax = plt.subplots(figsize=(10, 5))
ax.set_title("Distribution du Nombre d'Appel pour chaque type d'appel")
bars = ax.bar(x, [appel_jour, appel_soiree, appel_nuit, appel_inter], width, color='#F4CCCC')

# Adding labels, title, and legend
ax.set_xlabel("Types d'appel")
ax.set_ylabel('Nombre des appels')
ax.set_xticks(x)
ax.set_xticklabels(labels, rotation=45)
ax.legend(loc='upper right')

# Displaying the plot
plt.tight_layout()
plt.show()


No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


In [29]:
#Distribution de la Duration des Appels pour chaque type d'appel

# Assuming you have already loaded your data into a DataFrame called df

# Select the columns containing the durations for each time period
jour_duration = df['duree_appel_jour']
soiree_duration = df['duree_appel_soiree']
nuit_duration = df['duree_appel_nuit']
inter_duration = df['duree_appel_inter']
# Define pastel colors
colors = [(1, 0.75, 0.8), (0.75, 0.85, 1), (0.85, 1, 0.85),(0.7, 0.6, 0.8)]  # Pastel shades of pink, blue, and green

# Create subplots for each time period
plt.figure(figsize=(10, 6))

# Plot the distribution of durations for each time period without bars
sns.distplot(jour_duration, hist=False, color=colors[0], label='Jour')
sns.distplot(soiree_duration, hist=False, color=colors[1], label='Soiree')
sns.distplot(nuit_duration, hist=False, color=colors[2], label='Nuit')
sns.distplot(inter_duration, hist=False, color=colors[3], label='Internationale')

# Add labels and title
plt.xlabel('Duration')
plt.ylabel('Density')
plt.title('Distribution of Call Durations by Time Period')
plt.legend()

# Show the plot
plt.show()


C:\Users\chino\AppData\Local\Temp\ipykernel_8508\2556210571.py:17: UserWarning: 

`distplot` is a deprecated function and will be removed in seaborn v0.14.0.

Please adapt your code to use either `displot` (a figure-level function with
similar flexibility) or `kdeplot` (an axes-level function for kernel density plots).

For a guide to updating your code to use the new functions, please see
https://gist.github.com/mwaskom/de44147ed2974457ad6372750bbe5751

  sns.distplot(jour_duration, hist=False, color=colors[0], label='Jour')
C:\Users\chino\AppData\Local\Temp\ipykernel_8508\2556210571.py:18: UserWarning: 

`distplot` is a deprecated function and will be removed in seaborn v0.14.0.

Please adapt your code to use either `displot` (a figure-level function with
similar flexibility) or `kdeplot` (an axes-level function for kernel density plots).

For a guide to updating your code to use the new functions, please see
https://gist.github.com/mwaskom/de44147ed2974457ad6372750bbe5751

  sns.dis

In [27]:


#Distribution du Cout des Appels pour chaque type d'appel
count_data = pd.DataFrame({
    'Time_Period': ['Jour', 'Soiree', 'Nuit','Internationaux'],
    'Count': [df['cout_appel_jour'].sum(), df['cout_appel_soiree'].sum(), df['cout_appel_nuit'].sum(),df['cout_appel_inter'].sum()]
})

# Define pastel colors
colors = [(0.6, 0.4, 0.8)]  # Shades of pink, blue, and green

# Create grouped bar plot
plt.figure(figsize=(10, 6))
sns.barplot(x='Time_Period', y='Count', data=count_data, palette=colors)

# Add labels and title
plt.xlabel("Types d'appels")
plt.ylabel("Nombre d'Appels")
plt.title("Distribution du Cout des Appels pour chaque type d'appel")

# Show the plot
plt.show()




C:\Users\chino\AppData\Local\Temp\ipykernel_8508\1202556344.py:12: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x='Time_Period', y='Count', data=count_data, palette=colors)
C:\Users\chino\AppData\Local\Temp\ipykernel_8508\1202556344.py:12: UserWarning: 
The palette list has fewer values (1) than needed (4) and will cycle, which may produce an uninterpretable plot.
  sns.barplot(x='Time_Period', y='Count', data=count_data, palette=colors)


In [18]:
#  Distribution du nombre des réclamations
plt.figure(figsize=(8,8))
sns.countplot(x=df['nb_reclamation'], palette=['#FFC1CC','#FF66CC','#FF91A4'])
total = len(df['nb_reclamation'])
plt.xlabel("Nombre de réclamations ")
plt.ylabel("Nombre de clients")
plt.title("Distribution du nombre des réclamations")


C:\Users\chino\AppData\Local\Temp\ipykernel_8508\2860257450.py:3: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.countplot(x=df['nb_reclamation'], palette=['#FFC1CC','#FF66CC','#FF91A4'])
C:\Users\chino\AppData\Local\Temp\ipykernel_8508\2860257450.py:3: UserWarning: 
The palette list has fewer values (3) than needed (10) and will cycle, which may produce an uninterpretable plot.
  sns.countplot(x=df['nb_reclamation'], palette=['#FFC1CC','#FF66CC','#FF91A4'])


Text(0.5, 1.0, 'Distribution du nombre des réclamations')

In [19]:
# creation d'une fonction de constsruction de boites a moustaches et de maniere interactive 
def box_plot(column):
    sns.boxplot(y =df[column])
    plt.title('Boxplot of '+str(column))
    return plt.show()
#interact
interact(box_plot, column= numerical_columns)

interactive(children=(Dropdown(description='column', options=('age', 'nb_jours_abonne', 'duree_appel_jour', 'n…

<function __main__.box_plot(column)>

In [20]:
#ANALYSE BIVARIEE

In [21]:
#genre vs churn

In [22]:
#nb_reclamation vs churn


plt.figure(figsize=(8, 6))
sns.countplot(x='nb_reclamation', hue='churn', data=df)
plt.title('Répartition du churn selon nombre de réclamations')
plt.xlabel('Nombre de réclamation')
plt.ylabel('Nombre de clients')
plt.legend(title='Churn', loc='upper right')
plt.show()

In [23]:
sns.boxplot(x='churn', y='nb_reclamation', data=df, palette=['blue', 'orange'])
plt.xlabel('Churn')
plt.ylabel('Nombre de réclamation')
plt.show()

C:\Users\chino\AppData\Local\Temp\ipykernel_19392\2762798356.py:1: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(x='churn', y='nb_reclamation', data=df, palette=['blue', 'orange'])


In [24]:
#etat_civile vs churn


plt.figure(figsize=(8, 6))
sns.countplot(x='marie', hue='churn', data=df)
plt.title("Répartition du churn selon l'état civile")
plt.xlabel('marié')
plt.ylabel('Nombre de clients')
plt.legend(title='churn', loc='upper right')
plt.show()

In [25]:
#age vs churn

# Define age intervals
age_intervals = [(0, 18), (18, 30), (30, 50), (50, 70), (70, 120)]

# Create a new column in the DataFrame to represent age intervals
df['age_interval'] = pd.cut(df['age'], bins=[interval[0] for interval in age_intervals] + [np.inf], labels=[f'{interval[0]}-{interval[1]}' for interval in age_intervals])

# Plot the bar chart
plt.figure(figsize=(10, 6))
sns.countplot(x='age_interval', hue='churn', data=df, order=[f'{interval[0]}-{interval[1]}' for interval in age_intervals])
plt.title("Répartition du churn selon l'age")
plt.xlabel('Age')
plt.ylabel('Nombre de clients')
plt.legend(title='Churn', loc='upper right')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [26]:
sns.boxplot(x='churn', y='age', data=df, palette=['blue', 'orange'])
plt.xlabel('Churn')
plt.ylabel('Age ')
plt.show()

C:\Users\chino\AppData\Local\Temp\ipykernel_19392\1958056296.py:1: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(x='churn', y='age', data=df, palette=['blue', 'orange'])


In [27]:
#nb_jours_abonne vs churn

# Define subscription days intervals
abonnement_intervals = [(0, 50), (50, 100), (100, 150), (150, 200), (200, 250)]

# Create a new column in the DataFrame to represent subscription days intervals
df['subscription_interval'] = pd.cut(df['nb_jours_abonne'], bins=[interval[0] for interval in abonnement_intervals] + [np.inf], labels=[f'{interval[0]}-{interval[1]}' for interval in abonnement_intervals])

# Plot the bar chart using subscription_interval
plt.figure(figsize=(10, 6))
sns.countplot(x='subscription_interval', hue='churn', data=df, order=[f'{interval[0]}-{interval[1]}' for interval in abonnement_intervals])
plt.title("Répartition du churn selon la période d'abonnement")
plt.xlabel("Période d'abonnement en jours")
plt.ylabel('Nombre de clients')
plt.legend(title='Churn', loc='upper right')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [28]:
sns.boxplot(x='churn', y='nb_jours_abonne', data=df, palette=['blue', 'orange'])
plt.xlabel('Churn')
plt.ylabel("Période d'abonnement en jours")
plt.show()

C:\Users\chino\AppData\Local\Temp\ipykernel_19392\1100228588.py:1: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(x='churn', y='nb_jours_abonne', data=df, palette=['blue', 'orange'])


In [29]:
#active_msg_vocaux vs churn


plt.figure(figsize=(8, 6))
sns.countplot(x='active_msg_vocaux', hue='churn', data=df)
plt.title(" Répartition du churn selon l’activation des messages vocaux")
plt.xlabel('Active messages vocaux')
plt.ylabel('Nombre de clients')
plt.legend(title='churn', loc='upper right')
plt.show()

In [30]:
# Calculate the percentage distribution of churn based on the activation of voicemail messages
voicemail_churn_distribution = df.groupby('active_msg_vocaux')['churn'].value_counts(normalize=True).mul(100).rename('percentage').reset_index()

# Pivot the table to make it easier to read
voicemail_churn_pivot = voicemail_churn_distribution.pivot(index='active_msg_vocaux', columns='churn', values='percentage')

# Rename the columns for better clarity
voicemail_churn_pivot.columns = ['Not Churned (%)', 'Churned (%)']

# Display the table
print(voicemail_churn_pivot)


                   Not Churned (%)  Churned (%)
active_msg_vocaux                              
no                       83.556040    16.443960
yes                      92.278577     7.721423


In [31]:
#nb_msg_vocaux vs churn 
sns.boxplot(x='churn', y='nb_msg_vocaux', data=df, palette=['blue', 'orange'])
plt.xlabel('Churn')
plt.ylabel("Nombre de message vocaux")
plt.title(" Répartition du churn selon nombre de message vocaux")
plt.show()

C:\Users\chino\AppData\Local\Temp\ipykernel_19392\695860232.py:2: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(x='churn', y='nb_msg_vocaux', data=df, palette=['blue', 'orange'])


In [32]:
#cout_appel_jour vs churn
sns.boxplot(x='churn', y='cout_appel_jour', data=df, palette=['blue', 'orange'])
plt.xlabel('Churn')
plt.ylabel("Jour")
plt.title(" Répartition du churn selon le cout d'appel par jour")
plt.show()

C:\Users\chino\AppData\Local\Temp\ipykernel_19392\2676615220.py:2: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(x='churn', y='cout_appel_jour', data=df, palette=['blue', 'orange'])


In [26]:
#cout_appel_soiree vs churn
sns.boxplot(x='churn', y='cout_appel_soiree', data=df, palette=['blue', 'orange'])
plt.xlabel('Churn')
plt.ylabel("Soirée")
plt.title("Répartition du churn selon le cout d'appel par soirée")
plt.show()

C:\Users\chino\AppData\Local\Temp\ipykernel_8508\28414109.py:2: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(x='churn', y='cout_appel_soiree', data=df, palette=['blue', 'orange'])


In [25]:
#cout_appel_nuit vs churn
sns.boxplot(x='churn', y='cout_appel_nuit', data=df, palette=['blue', 'orange'])
plt.xlabel('Churn')
plt.ylabel("Nuit")
plt.title("Répartition du churn selon le cout d'appel par nuit")
plt.show()

C:\Users\chino\AppData\Local\Temp\ipykernel_8508\1306726788.py:2: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(x='churn', y='cout_appel_nuit', data=df, palette=['blue', 'orange'])


In [21]:
#cout_appel_intern vs churn
sns.boxplot(x='churn', y='cout_appel_inter', data=df, palette=['blue', 'orange'])
plt.xlabel('Churn')
plt.ylabel("International")
plt.title("Répartition du churn selon le cout d'appels internationaux")
plt.show()

C:\Users\chino\AppData\Local\Temp\ipykernel_8508\1803368150.py:2: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(x='churn', y='cout_appel_inter', data=df, palette=['blue', 'orange'])


In [24]:
import seaborn as sns
import matplotlib.pyplot as plt

colors = ['#225ead', '#e1cbd7']
sns.lmplot(x='duree_appel_jour', y='duree_appel_soiree', data=df, hue='churn', palette=colors)
plt.title("Relation entre la durée d'appel de jour et de soirée pour les Clients churn et non churn")
plt.show()


In [36]:
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming df is your DataFrame containing the data
colors = ['#225ead', '#e1cbd7']
sns.lmplot(x='duree_appel_jour', y='duree_appel_nuit', data=df, hue='churn', palette='pastel')
plt.title("Relation entre la durée d'appel de jour et de soirée pour les Clients churn et non churn")
plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming df is your DataFrame containing the data
colors = ['#ff9999', '#66b3ff'] 
sns.lmplot(x='duree_appel_soiree', y='duree_appel_nuit', data=df, hue='churn', palette='pastel')
plt.title("Relation entre la durée d'appel de jour et de soirée pour les Clients churn et non churn")
plt.show()


In [37]:
df.isna().sum()

id_client                 0
genre                     0
age                       0
marie                     0
num_tel                   0
nb_jours_abonne          10
duree_appel_jour          0
nb_appel_jour             7
cout_appel_jour           0
duree_appel_soiree        0
nb_appel_soiree           6
cout_appel_soiree         0
duree_appel_nuit          0
nb_appel_nuit            11
cout_appel_nuit           0
duree_appel_inter         0
nb_appel_inter           12
cout_appel_inter          0
active_msg_vocaux         8
nb_msg_vocaux             0
nb_reclamation           12
churn                     4
offer_type                0
age_interval              0
subscription_interval    10
dtype: int64

In [38]:
print(df.duplicated().sum())

0


In [39]:
#Prétrqitement des données

In [40]:
for x in df.columns :
    print(x, len(df[x].unique()))

id_client 5000
genre 2
age 62
marie 2
num_tel  33
nb_jours_abonne 219
duree_appel_jour 1961
nb_appel_jour 124
cout_appel_jour 1961
duree_appel_soiree 1879
nb_appel_soiree 127
cout_appel_soiree 1659
duree_appel_nuit 1853
nb_appel_nuit 132
cout_appel_nuit 1028
duree_appel_inter 170
nb_appel_inter 22
cout_appel_inter 170
active_msg_vocaux 3
nb_msg_vocaux 48
nb_reclamation 11
churn 3
offer_type 40
age_interval 4
subscription_interval 6


In [41]:
df = df.drop(df.columns[[0, 4]], axis=1)
df.head()


,genre,age,marie,nb_jours_abonne,duree_appel_jour,nb_appel_jour,cout_appel_jour,duree_appel_soiree,nb_appel_soiree,cout_appel_soiree,...,duree_appel_inter,nb_appel_inter,cout_appel_inter,active_msg_vocaux,nb_msg_vocaux,nb_reclamation,churn,offer_type,age_interval,subscription_interval
0,femme,37,yes,128.0,265.1,110.0,45.07,197.4,99.0,16.78,...,10.0,3.0,2.70,yes,25,1.0,False,Hayya,30-50,100-150
1,homme,46,no,107.0,161.6,123.0,27.47,195.5,103.0,16.62,...,13.7,3.0,3.70,yes,26,1.0,False,PRE - 1=11,30-50,100-150
2,homme,50,no,137.0,243.4,114.0,41.38,121.2,110.0,10.30,...,12.2,5.0,3.29,no,0,0.0,False,PRE - 900 bonus,30-50,100-150
3,homme,78,yes,84.0,299.4,71.0,50.90,61.9,88.0,5.26,...,6.6,7.0,1.78,no,0,2.0,False,PRE - AHLA,70-120,50-100
4,femme,75,yes,75.0,166.7,113.0,28.34,148.3,122.0,12.61,...,10.1,3.0,2.73,no,0,3.0,False,PRE - Binetna,70-120,50-100


In [42]:
#Vérifier les valeurs nuls 
df.isnull().sum()

genre                     0
age                       0
marie                     0
nb_jours_abonne          10
duree_appel_jour          0
nb_appel_jour             7
cout_appel_jour           0
duree_appel_soiree        0
nb_appel_soiree           6
cout_appel_soiree         0
duree_appel_nuit          0
nb_appel_nuit            11
cout_appel_nuit           0
duree_appel_inter         0
nb_appel_inter           12
cout_appel_inter          0
active_msg_vocaux         8
nb_msg_vocaux             0
nb_reclamation           12
churn                     4
offer_type                0
age_interval              0
subscription_interval    10
dtype: int64

In [43]:
df['nb_jours_abonne'].value_counts()

nb_jours_abonne
90.0     65
87.0     59
105.0    57
93.0     57
112.0    56
         ..
215.0     1
238.0     1
216.0     1
208.0     1
233.0     1
Name: count, Length: 218, dtype: int64

In [44]:
df['nb_jours_abonne'] = df['nb_jours_abonne'].fillna(df["nb_jours_abonne"].max())

In [45]:
df['nb_reclamation'].value_counts()

nb_reclamation
1.0    1781
2.0    1126
0.0    1020
3.0     663
4.0     252
5.0      95
6.0      34
7.0      13
9.0       2
8.0       2
Name: count, dtype: int64

In [46]:
df['nb_reclamation'] = df['nb_reclamation'].fillna(1.0)

In [47]:

df['nb_appel_jour'] = df['nb_appel_jour'].fillna(df["nb_appel_jour"].max())
df['nb_appel_soiree'] = df['nb_appel_soiree'].fillna(df["nb_appel_soiree"].max())
df['nb_appel_nuit'] = df['nb_appel_nuit'].fillna(df["nb_appel_nuit"].max())

In [48]:
mean_value = df['nb_appel_inter'].mean()
mean_value = round(mean_value)
mean_value

4

In [49]:
df["nb_appel_inter"] = df['nb_appel_inter'].fillna(mean_value)

In [50]:
df['active_msg_vocaux'].fillna(method ="ffill", inplace = True)

C:\Users\chino\AppData\Local\Temp\ipykernel_19392\37469121.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['active_msg_vocaux'].fillna(method ="ffill", inplace = True)
C:\Users\chino\AppData\Local\Temp\ipykernel_19392\37469121.py:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['active_msg_vocaux'].fillna(method ="ffill", inplace = True)


In [51]:
df['churn'].value_counts()

churn
False    4289
True      707
Name: count, dtype: int64

In [52]:
df['churn'].fillna(method ="bfill", inplace = True)

C:\Users\chino\AppData\Local\Temp\ipykernel_19392\929120905.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['churn'].fillna(method ="bfill", inplace = True)
C:\Users\chino\AppData\Local\Temp\ipykernel_19392\929120905.py:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['churn'].fillna(method ="bfill", inplace = True)


In [53]:
df.isnull().sum()

genre                     0
age                       0
marie                     0
nb_jours_abonne           0
duree_appel_jour          0
nb_appel_jour             0
cout_appel_jour           0
duree_appel_soiree        0
nb_appel_soiree           0
cout_appel_soiree         0
duree_appel_nuit          0
nb_appel_nuit             0
cout_appel_nuit           0
duree_appel_inter         0
nb_appel_inter            0
cout_appel_inter          0
active_msg_vocaux         0
nb_msg_vocaux             0
nb_reclamation            0
churn                     0
offer_type                0
age_interval              0
subscription_interval    10
dtype: int64

In [54]:
#Vérifier s'il y a des doublons 
print(df.duplicated().sum())

0


In [55]:
df.dtypes

genre                      object
age                         int64
marie                      object
nb_jours_abonne           float64
duree_appel_jour          float64
nb_appel_jour             float64
cout_appel_jour           float64
duree_appel_soiree        float64
nb_appel_soiree           float64
cout_appel_soiree         float64
duree_appel_nuit          float64
nb_appel_nuit             float64
cout_appel_nuit           float64
duree_appel_inter         float64
nb_appel_inter            float64
cout_appel_inter          float64
active_msg_vocaux          object
nb_msg_vocaux               int64
nb_reclamation            float64
churn                      object
offer_type                 object
age_interval             category
subscription_interval    category
dtype: object

In [56]:
label_encoder = LabelEncoder()
df['genre'] = label_encoder.fit_transform(df['genre'])
df['marie'] = label_encoder.fit_transform(df['marie'])
df['active_msg_vocaux'] = label_encoder.fit_transform(df['active_msg_vocaux'])
df['churn'] = label_encoder.fit_transform(df['churn'])

In [57]:
# Calculer la moyenne de 'encoded_churn' pour chaque type d'offre
target_mean_df = df.groupby('offer_type')['churn'].mean().reset_index()
offer_type_mapping = dict(zip(target_mean_df['offer_type'], target_mean_df['churn']))

df['offer_type'] = df['offer_type'].map(offer_type_mapping)


In [58]:
# Save the DataFrame to a CSV file
df.to_csv('encoded_dataset.csv', index=False)  # Set index=False to avoid saving the index
df

,genre,age,marie,nb_jours_abonne,duree_appel_jour,nb_appel_jour,cout_appel_jour,duree_appel_soiree,nb_appel_soiree,cout_appel_soiree,...,duree_appel_inter,nb_appel_inter,cout_appel_inter,active_msg_vocaux,nb_msg_vocaux,nb_reclamation,churn,offer_type,age_interval,subscription_interval
0,0,37,1,128.0,265.1,110.0,45.07,197.4,99.0,16.78,...,10.0,3.0,2.70,1,25,1.0,0,0.166667,30-50,100-150
1,1,46,0,107.0,161.6,123.0,27.47,195.5,103.0,16.62,...,13.7,3.0,3.70,1,26,1.0,0,0.139344,30-50,100-150
2,1,50,0,137.0,243.4,114.0,41.38,121.2,110.0,10.30,...,12.2,5.0,3.29,0,0,0.0,0,0.122137,30-50,100-150
3,1,78,1,84.0,299.4,71.0,50.90,61.9,88.0,5.26,...,6.6,7.0,1.78,0,0,2.0,0,0.144860,70-120,50-100
4,0,75,1,75.0,166.7,113.0,28.34,148.3,122.0,12.61,...,10.1,3.0,2.73,0,0,3.0,0,0.116732,70-120,50-100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,0,65,0,50.0,235.7,127.0,40.07,223.0,126.0,18.96,...,9.9,5.0,2.67,1,40,2.0,0,0.122137,50-70,0-50
4996,1,46,1,152.0,184.2,90.0,31.31,256.8,73.0,21.83,...,14.7,2.0,3.97,0,0,3.0,1,0.144860,30-50,150-200
4997,0,77,0,61.0,140.6,89.0,23.90,172.8,128.0,14.69,...,13.6,4.0,3.67,0,0,1.0,0,0.116732,70-120,50-100
4998,1,62,0,109.0,188.8,67.0,32.10,171.7,92.0,14.59,...,8.5,6.0,2.30,0,0,0.0,0,0.135952,50-70,100-150


In [59]:
# Select only numerical columns for correlation calculation
numeric_df = df.select_dtypes(include=['number'])

# Calculate the correlation matrix
correlation_matrix = numeric_df.corr()

# Set up the matplotlib figure
plt.figure(figsize=(10, 8))

# Plot the heatmap
sns.heatmap(correlation_matrix, annot=True, cmap='Pastel2', fmt=".2f")

# Add title and adjust layout
plt.title('Matrice de Corrélation')
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
# Select only numeric columns for correlation calculation
numeric_df = df.select_dtypes(include=['number'])

# Calculate the correlation matrix
corr = numeric_df.corr()[['churn']]

# Set up the matplotlib figure
plt.figure(figsize=(10, 8))

# Plot the heatmap
sns.heatmap(corr, cmap='crest', annot=True, center=0.0)

# Add title and adjust layout
plt.title('Corrélation entre "churn" et les autres variables', fontsize=16)
plt.tight_layout()

# Show the plot
plt.show()


In [ ]:
#MODELISATION

In [ ]:
data = pd.read_csv('encoded_TT_Churn.csv')
data

,age,marie,genre,nb_jours_abonne,active_msg_vocaux,nb_msg_vocaux,nb_appel_jour,duree_appel_jour,cout_appel_jour,nb_appel_soiree,...,cout_appel_soiree,nb_appel_nuit,duree_appel_nuit,cout_appel_nuit,nb_appel_inter,duree_appel_inter,cout_appel_inter,nb_reclamation,churn,offer_type
0,19,0,0,34,0,0,80,128.8,21.90,93,...,17.74,103,202.1,9.09,7.0,14.0,3.78,1,0,0.153425
1,19,0,0,38,0,0,110,96.9,16.47,75,...,20.52,69,141.0,6.35,3.0,10.8,2.92,0,0,0.134409
2,19,0,0,43,1,36,68,93.1,15.83,83,...,20.92,109,94.9,4.27,8.0,8.7,2.35,1,0,0.146341
3,19,0,0,48,1,37,81,138.3,23.51,74,...,15.95,113,158.0,7.11,1.0,11.6,3.13,2,0,0.173709
4,19,0,0,60,0,0,77,207.3,35.24,105,...,17.67,89,108.2,4.87,5.0,12.9,3.48,1,0,0.146341
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,80,1,1,71,0,0,130,123.8,21.05,91,...,19.25,96,213.9,9.63,7.0,8.8,2.38,2,0,0.094828
4996,80,1,1,82,0,0,112,172.4,29.31,89,...,11.19,78,182.8,8.23,9.0,8.8,2.38,1,0,0.132075
4997,80,1,1,137,0,0,107,75.8,12.89,98,...,21.84,118,204.4,9.20,3.0,10.5,2.84,2,0,0.163265
4998,80,1,1,139,1,23,129,157.6,26.79,96,...,21.00,112,259.2,11.66,2.0,13.7,3.70,0,0,0.122137


In [ ]:
data.isna().sum()

age                   0
marie                 0
genre                 0
nb_jours_abonne       0
active_msg_vocaux     0
nb_msg_vocaux         0
nb_appel_jour         0
duree_appel_jour      0
cout_appel_jour       0
nb_appel_soiree       0
duree_appel_soiree    0
cout_appel_soiree     0
nb_appel_nuit         0
duree_appel_nuit      0
cout_appel_nuit       0
nb_appel_inter        0
duree_appel_inter     0
cout_appel_inter      0
nb_reclamation        0
churn                 0
offer_type            0
dtype: int64

In [ ]:
#Extraction des variables prédictives et cible
x=data.drop(["churn"], axis =1)
y=data["churn"]
x

,age,marie,genre,nb_jours_abonne,active_msg_vocaux,nb_msg_vocaux,nb_appel_jour,duree_appel_jour,cout_appel_jour,nb_appel_soiree,duree_appel_soiree,cout_appel_soiree,nb_appel_nuit,duree_appel_nuit,cout_appel_nuit,nb_appel_inter,duree_appel_inter,cout_appel_inter,nb_reclamation,offer_type
0,19,0,0,34,0,0,80,128.8,21.90,93,208.7,17.74,103,202.1,9.09,7.0,14.0,3.78,1,0.153425
1,19,0,0,38,0,0,110,96.9,16.47,75,241.4,20.52,69,141.0,6.35,3.0,10.8,2.92,0,0.134409
2,19,0,0,43,1,36,68,93.1,15.83,83,246.1,20.92,109,94.9,4.27,8.0,8.7,2.35,1,0.146341
3,19,0,0,48,1,37,81,138.3,23.51,74,187.6,15.95,113,158.0,7.11,1.0,11.6,3.13,2,0.173709
4,19,0,0,60,0,0,77,207.3,35.24,105,207.9,17.67,89,108.2,4.87,5.0,12.9,3.48,1,0.146341
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,80,1,1,71,0,0,130,123.8,21.05,91,226.5,19.25,96,213.9,9.63,7.0,8.8,2.38,2,0.094828
4996,80,1,1,82,0,0,112,172.4,29.31,89,131.7,11.19,78,182.8,8.23,9.0,8.8,2.38,1,0.132075
4997,80,1,1,137,0,0,107,75.8,12.89,98,256.9,21.84,118,204.4,9.20,3.0,10.5,2.84,2,0.163265
4998,80,1,1,139,1,23,129,157.6,26.79,96,247.0,21.00,112,259.2,11.66,2.0,13.7,3.70,0,0.122137


In [ ]:
np.random.seed(0)
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2, random_state=15, stratify=y)
print("Train set shape:", x_train.shape, y_train.shape)
print("Test set shape:", x_test.shape, y_test.shape)

Train set shape: (4000, 20) (4000,)
Test set shape: (1000, 20) (1000,)


In [ ]:
#Standarisation 

sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [ ]:

#RandomForest

# Define cross-validation strategy (e.g., using KFold with 5 folds)
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Grid Search to improve accuracy with cross-validation
parameters = {'n_estimators': [100, 150, 200,250], 'max_depth': [3, 5, 7, 9, 11,18]}
rf_model = RandomForestClassifier()
clf = GridSearchCV(estimator=rf_model, param_grid=parameters, n_jobs=-1, cv=cv)

# Fitting the model
clf.fit(x_train, y_train)

# Getting the best parameters and score
print("Best Parameters for Random Forest:")
print(clf.best_params_)
print("Best Score:", clf.best_score_)

# Train Final Model with Best Parameters
final_modelRF = RandomForestClassifier(n_estimators=clf.best_params_['n_estimators'],
                                      max_depth=clf.best_params_['max_depth'])
final_modelRF.fit(x_train, y_train)

# Evaluate Final Model
# Make predictions on the test set
y_pred = final_modelRF.predict(x_test)

# Compute evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
print("Testing Accuracy:", accuracy)

# Classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Compute confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:")
print(conf_matrix)

Best Parameters for Random Forest:
{'max_depth': 18, 'n_estimators': 200}
Best Score: 0.92925
Testing Accuracy: 0.927

Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.99      0.96       858
           1       0.94      0.52      0.67       142

    accuracy                           0.93      1000
   macro avg       0.93      0.76      0.81      1000
weighted avg       0.93      0.93      0.92      1000


Confusion Matrix:
[[853   5]
 [ 68  74]]


In [ ]:

# Compute confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)

# Plot confusion matrix

plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='twilight', cbar=False,linewidths=1, linecolor='gray')
y_labels =['Non Churn(0)','Churn(1)']
plt.title('Matrice de Confusion')
plt.xlabel('Classe Prédite')
plt.ylabel('Classe Actuelle')

plt.gca().set_xticklabels(y_labels)
plt.gca().set_yticklabels(y_labels)
plt.show()


In [ ]:

# Compute precision-recall curve
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc
precision, recall, _ = precision_recall_curve(y_test, final_modelRF.predict_proba(x_test)[:, 1])
pr_auc = auc(recall, precision)

# Plot precision-recall curve
plt.figure(figsize=(8, 6))
plt.plot(recall, precision, label=f'PR Curve (AUC = {pr_auc:.2f})', color='#AEC6CF')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend(loc='lower left')
plt.show()



In [ ]:

# Compute learning curve
from sklearn.model_selection import learning_curve
train_sizes, train_scores, valid_scores = learning_curve(final_modelRF, x_train, y_train, train_sizes=np.linspace(0.1, 1.0, 10), cv=5)

# Plot learning curve
plt.figure(figsize=(8, 6))
plt.plot(train_sizes, np.mean(train_scores, axis=1), label='Training Score',color='#FFB6C1')
plt.plot(train_sizes, np.mean(valid_scores, axis=1), label='Testing Score',color='#AEC6CF')
plt.xlabel('Training Examples')
plt.ylabel('Score')
plt.title('Learning Curve')
plt.legend(loc='best')
plt.show()


In [ ]:
#Adaboost model
import pickle
# Define cross-validation strategy (e.g., using KFold with 5 folds)
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Grid Search to improve accuracy with cross-validation
parameters = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 1.0]}
adaboost_model = AdaBoostClassifier()
clf = GridSearchCV(estimator=adaboost_model, param_grid=parameters, n_jobs=-1, cv=cv)

# Fitting the model
clf.fit(x_train, y_train)

# Getting the best parameters and score
print("Best Parameters for AdaBoost:")
print(clf.best_params_)
print("Best Score:", clf.best_score_)

# Train Final Model with Best Parameters
final_model_adaboost = AdaBoostClassifier(n_estimators=clf.best_params_['n_estimators'], 
                                          learning_rate=clf.best_params_['learning_rate'])
final_model_adaboost.fit(x_train, y_train)

# Save the trained model to a file
with open("final_model_adaboost.pkl", "wb") as f:
    pickle.dump(final_model_adaboost, f)

# Evaluate Final Model
# Make predictions on the test set
y_pred_adaboost = final_model_adaboost.predict(x_test)

# Compute evaluation metrics
accuracy_adaboost = accuracy_score(y_test, y_pred_adaboost)
print("Testing Accuracy for AdaBoost:", accuracy_adaboost)

# Classification report
print("\nClassification Report for AdaBoost:")
print(classification_report(y_test, y_pred_adaboost))

# Compute AUC
auc_adaboost = roc_auc_score(y_test, y_pred_adaboost)
print("AUC for AdaBoost:", auc_adaboost)

# Compute confusion matrix
conf_matrix_adaboost = confusion_matrix(y_test, y_pred_adaboost)
print("\nConfusion Matrix for AdaBoost:")
print(conf_matrix_adaboost)


c:\Users\chino\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Best Parameters for AdaBoost:
{'learning_rate': 1.0, 'n_estimators': 150}
Best Score: 0.8727500000000001


c:\Users\chino\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Testing Accuracy for AdaBoost: 0.887

Classification Report for AdaBoost:
              precision    recall  f1-score   support

           0       0.90      0.98      0.94       858
           1       0.70      0.35      0.47       142

    accuracy                           0.89      1000
   macro avg       0.80      0.66      0.70      1000
weighted avg       0.87      0.89      0.87      1000

AUC for AdaBoost: 0.6638185757904067

Confusion Matrix for AdaBoost:
[[837  21]
 [ 92  50]]


In [ ]:

# Compute confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred_adaboost)

# Plot confusion matrix

plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='twilight', cbar=False,linewidths=1, linecolor='gray')
y_labels =['Non Churn(0)','Churn(1)']
plt.title('Matrice de Confusion')
plt.xlabel('Classe Prédite')
plt.ylabel('Classe Actuelle')

plt.gca().set_xticklabels(y_labels)
plt.gca().set_yticklabels(y_labels)
plt.show()


In [ ]:
# Compute precision-recall curve
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc
precision, recall, _ = precision_recall_curve(y_test, final_model_adaboost.predict_proba(x_test)[:, 1])
pr_auc = auc(recall, precision)

# Plot precision-recall curve
plt.figure(figsize=(8, 6))
plt.plot(recall, precision, label=f'PR Curve (AUC = {pr_auc:.2f})', color='#AEC6CF')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend(loc='lower left')
plt.show()

In [ ]:
#LogisticRegression model
# Define cross-validation strategy (e.g., using KFold with 5 folds)
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Grid Search to improve accuracy with cross-validation
parameters = {'C': [0.001, 0.01, 0.1, 1, 10, 100]}
logistic_model = LogisticRegression()
clf = GridSearchCV(estimator=logistic_model, param_grid=parameters, n_jobs=-1, cv=cv)

# Fitting the model
clf.fit(x_train, y_train)

# Getting the best parameters and score
print("Best Parameters for Logistic Regression:")
print(clf.best_params_)
print("Best Score:", clf.best_score_)

# Train Final Model with Best Parameters
final_modelLR = LogisticRegression(C=clf.best_params_['C'])
final_modelLR.fit(x_train, y_train)

# Evaluate Final Model
# Make predictions on the test set
y_pred = final_modelLR.predict(x_test)

# Compute evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
print("Testing Accuracy:", accuracy)

# Classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Compute AUC
auc = roc_auc_score(y_test, y_pred)
print("AUC:", auc)

# Compute confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:")
print(conf_matrix)

Best Parameters for Logistic Regression:
{'C': 1}
Best Score: 0.8649999999999999
Testing Accuracy: 0.863

Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.99      0.93       858
           1       0.61      0.10      0.17       142

    accuracy                           0.86      1000
   macro avg       0.74      0.54      0.55      1000
weighted avg       0.83      0.86      0.82      1000

AUC: 0.544051019403132

Confusion Matrix:
[[849   9]
 [128  14]]


In [ ]:
# Compute confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)

# Plot confusion matrix

plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='twilight', cbar=False,linewidths=1, linecolor='gray')
y_labels =['Non Churn(0)','Churn(1)']
plt.title('Matrice de Confusion')
plt.xlabel('Classe Prédite')
plt.ylabel('Classe Actuelle')

plt.gca().set_xticklabels(y_labels)
plt.gca().set_yticklabels(y_labels)
plt.show()


In [ ]:
# Compute learning curve
from sklearn.model_selection import learning_curve
train_sizes, train_scores, valid_scores = learning_curve(final_modelLR, x_train, y_train, train_sizes=np.linspace(0.1, 1.0, 10), cv=5)

# Plot learning curve
plt.figure(figsize=(8, 6))
plt.plot(train_sizes, np.mean(train_scores, axis=1), label='Training Score',color='#FFB6C1')
plt.plot(train_sizes, np.mean(valid_scores, axis=1), label='Testing Score',color='#AEC6CF')
plt.xlabel('Training Examples')
plt.ylabel('Score')
plt.title('Learning Curve')
plt.legend(loc='best')
plt.show()

C:\Users\chino\AppData\Local\Temp\ipykernel_19852\1157593320.py:6: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure(figsize=(8, 6))


In [ ]:
#xgboost model
import pickle
from sklearn.model_selection import KFold, GridSearchCV
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, confusion_matrix

# Define cross-validation strategy (e.g., using KFold with 5 folds)
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Grid Search to improve accuracy with cross-validation
parameters = {'n_estimators': [100, 150, 200], 'max_depth': [3, 5, 7]}
xgb_model = XGBClassifier()
clf = GridSearchCV(estimator=xgb_model, param_grid=parameters, n_jobs=-1, cv=cv)

# Fitting the model
clf.fit(x_train, y_train)

# Getting the best parameters and score
print("Best Parameters for XGBoost:")
print(clf.best_params_)
print("Best Score:", clf.best_score_)

# Train Final Model with Best Parameters
final_modelxgb = XGBClassifier(n_estimators=clf.best_params_['n_estimators'], 
                            max_depth=clf.best_params_['max_depth'])
final_modelxgb.fit(x_train, y_train)

# Save the trained model to a file
with open("final_modelxgb.pkl", "wb") as f:
    pickle.dump(final_modelxgb, f)

# Evaluate Final Model
# Make predictions on the test set
y_pred = final_modelxgb.predict(x_test)

# Compute evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
print("Testing Accuracy:", accuracy)

# Classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Compute AUC
auc = roc_auc_score(y_test, y_pred)
print("AUC:", auc)

# Compute confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:")
print(conf_matrix)


Best Parameters for XGBoost:
{'max_depth': 7, 'n_estimators': 150}
Best Score: 0.92875
Testing Accuracy: 0.924

Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.99      0.96       858
           1       0.90      0.52      0.66       142

    accuracy                           0.92      1000
   macro avg       0.91      0.76      0.81      1000
weighted avg       0.92      0.92      0.92      1000

AUC: 0.7559013756196855

Confusion Matrix:
[[850   8]
 [ 68  74]]


In [ ]:
# Compute confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)

# Plot confusion matrix

plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='twilight', cbar=False,linewidths=1, linecolor='gray')
y_labels =['Non Churn(0)','Churn(1)']
plt.title('Matrice de Confusion')
plt.xlabel('Classe Prédite')
plt.ylabel('Classe Actuelle')

plt.gca().set_xticklabels(y_labels)
plt.gca().set_yticklabels(y_labels)
plt.show()


In [ ]:
# Compute ROC curve
from sklearn.metrics import roc_curve
fpr, tpr, _ = roc_curve(y_test, final_modelxgb.predict_proba(x_test)[:, 1])

# Plot ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label='ROC Curve', color='b')
plt.plot([0, 1], [0, 1], linestyle='--', color='r', label='Random Guess')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc='lower right')
plt.show()

In [ ]:
# Compute learning curve
from sklearn.model_selection import learning_curve
train_sizes, train_scores, valid_scores = learning_curve(final_modelxgb, x_train, y_train, train_sizes=np.linspace(0.1, 1.0, 10), cv=5)

# Plot learning curve
plt.figure(figsize=(8, 6))
plt.plot(train_sizes, np.mean(train_scores, axis=1), label='Training Score',color='#FFB6C1')
plt.plot(train_sizes, np.mean(valid_scores, axis=1), label='Testing Score',color='#AEC6CF')
plt.xlabel('Training Examples')
plt.ylabel('Score')
plt.title('Learning Curve')
plt.legend(loc='best')
plt.show()

In [ ]:
#KNN model
# Define cross-validation strategy (e.g., using KFold with 5 folds)
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Grid Search to improve accuracy with cross-validation
parameters = {'n_neighbors': [3, 5, 7, 9, 11]}
knn_model = KNeighborsClassifier()
clf = GridSearchCV(estimator=knn_model, param_grid=parameters, n_jobs=-1, cv=cv)

# Fitting the model
clf.fit(x_train, y_train)

# Getting the best parameters and score
print("Best Parameters for K-Nearest Neighbors:")
print(clf.best_params_)
print("Best Score:", clf.best_score_)

# Train Final Model with Best Parameters
final_modelknn = KNeighborsClassifier(n_neighbors=clf.best_params_['n_neighbors'])
final_modelknn.fit(x_train, y_train)

# Evaluate Final Model
# Make predictions on the test set
y_pred = final_modelknn.predict(x_test)

# Compute evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
print("Testing Accuracy:", accuracy)

# Classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Compute AUC
auc = roc_auc_score(y_test, y_pred)
print("AUC:", auc)

# Compute confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:")
print(conf_matrix)

Best Parameters for K-Nearest Neighbors:
{'n_neighbors': 7}
Best Score: 0.88225
Testing Accuracy: 0.87

Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.99      0.93       858
           1       0.68      0.16      0.26       142

    accuracy                           0.87      1000
   macro avg       0.78      0.57      0.60      1000
weighted avg       0.85      0.87      0.83      1000

AUC: 0.5745756590827014

Confusion Matrix:
[[847  11]
 [119  23]]


In [ ]:
# Compute confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)

# Plot confusion matrix

plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='twilight', cbar=False,linewidths=1, linecolor='gray')
y_labels =['Non Churn(0)','Churn(1)']
plt.title('Matrice de Confusion')
plt.xlabel('Classe Prédite')
plt.ylabel('Classe Actuelle')

plt.gca().set_xticklabels(y_labels)
plt.gca().set_yticklabels(y_labels)
plt.show()


In [ ]:
# Compute learning curve
from sklearn.model_selection import learning_curve
train_sizes, train_scores, valid_scores = learning_curve(final_modelknn, x_train, y_train, train_sizes=np.linspace(0.1, 1.0, 10), cv=5)

# Plot learning curve
plt.figure(figsize=(8, 6))
plt.plot(train_sizes, np.mean(train_scores, axis=1), label='Training Score',color='#FFB6C1')
plt.plot(train_sizes, np.mean(valid_scores, axis=1), label='Testing Score',color='#AEC6CF')
plt.xlabel('Training Examples')
plt.ylabel('Score')
plt.title('Learning Curve')
plt.legend(loc='best')
plt.show()

In [ ]:
#Decision Tree model
# Define cross-validation strategy (e.g., using KFold with 5 folds)
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Grid Search to improve accuracy with cross-validation
parameters = {'max_depth': [3, 5, 7, 9, 11]}
dt_model = DecisionTreeClassifier()
clf = GridSearchCV(estimator=dt_model, param_grid=parameters, n_jobs=-1, cv=cv)

# Fitting the model
clf.fit(x_train, y_train)

# Getting the best parameters and score
print("Best Parameters for Decision Tree:")
print(clf.best_params_)
print("Best Score:", clf.best_score_)

# Train Final Model with Best Parameters
final_modelDT = DecisionTreeClassifier(max_depth=clf.best_params_['max_depth'])
final_modelDT.fit(x_train, y_train)

# Evaluate Final Model
# Make predictions on the test set
y_pred = final_modelDT.predict(x_test)

# Compute evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
print("Testing Accuracy:", accuracy)

# Classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Compute AUC
auc = roc_auc_score(y_test, y_pred)
print("AUC:", auc)

# Compute confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:")
print(conf_matrix)

Best Parameters for Decision Tree:
{'max_depth': 5}
Best Score: 0.91525
Testing Accuracy: 0.912

Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.98      0.95       858
           1       0.83      0.48      0.61       142

    accuracy                           0.91      1000
   macro avg       0.87      0.73      0.78      1000
weighted avg       0.91      0.91      0.90      1000

AUC: 0.7312781115598017

Confusion Matrix:
[[844  14]
 [ 74  68]]


In [ ]:
# Compute confusion matrix
conf_matrix_DT = confusion_matrix(y_test, y_pred)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix_DT, annot=True, fmt='d', cmap='twilight', cbar=False, linewidths=1, linecolor='gray')
y_labels = ['Non Churn(0)', 'Churn(1)']
plt.title('Matrice de Confusion')
plt.xlabel('Classe Prédite')
plt.ylabel('Classe Actuelle')

plt.gca().set_xticklabels(y_labels)
plt.gca().set_yticklabels(y_labels)
plt.show()


In [ ]:
# Compute learning curve
from sklearn.model_selection import learning_curve
train_sizes_dt, train_scores_dt, valid_scores_dt = learning_curve(final_modelDT, x_train, y_train, train_sizes=np.linspace(0.1, 1.0, 10), cv=5)

# Plot learning curve
plt.figure(figsize=(8, 6))
plt.plot(train_sizes_dt, np.mean(train_scores_dt, axis=1), label='Training Score', color='#FFB6C1')
plt.plot(train_sizes_dt, np.mean(valid_scores_dt, axis=1), label='Testing Score', color='#AEC6CF')
plt.xlabel('Training Examples')
plt.ylabel('Score')
plt.title('Learning Curve - Decision Tree')
plt.legend(loc='best')
plt.show()


In [ ]:
#Naive Bayes model
# Initialize Naive Bayes classifier model
nb_model = GaussianNB()

# Fitting the model
nb_model.fit(x_train, y_train)

# Evaluate Final Model
# Make predictions on the test set
y_pred = nb_model.predict(x_test)

# Compute evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
print("Testing Accuracy:", accuracy)

# Classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Compute AUC
auc = roc_auc_score(y_test, y_pred)
print("AUC:", auc)

# Compute confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:")
print(conf_matrix)

Testing Accuracy: 0.89

Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.96      0.94       858
           1       0.67      0.44      0.53       142

    accuracy                           0.89      1000
   macro avg       0.79      0.70      0.74      1000
weighted avg       0.88      0.89      0.88      1000

AUC: 0.7037657178502249

Confusion Matrix:
[[827  31]
 [ 79  63]]


In [ ]:
# Compute confusion matrix
conf_matrix_NB = confusion_matrix(y_test, y_pred)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix_NB, annot=True, fmt='d', cmap='twilight', cbar=False, linewidths=1, linecolor='gray')
y_labels = ['Non Churn(0)', 'Churn(1)']
plt.title('Matrice de Confusion')
plt.xlabel('Classe Prédite')
plt.ylabel('Classe Actuelle')

plt.gca().set_xticklabels(y_labels)
plt.gca().set_yticklabels(y_labels)
plt.show()


In [ ]:
# Compute learning curve
from sklearn.model_selection import learning_curve
train_sizes_dt, train_scores_dt, valid_scores_dt = learning_curve(nb_model, x_train, y_train, train_sizes=np.linspace(0.1, 1.0, 10), cv=5)

# Plot learning curve
plt.figure(figsize=(8, 6))
plt.plot(train_sizes_dt, np.mean(train_scores_dt, axis=1), label='Training Score', color='#FFB6C1')
plt.plot(train_sizes_dt, np.mean(valid_scores_dt, axis=1), label='Testing Score', color='#AEC6CF')
plt.xlabel('Training Examples')
plt.ylabel('Score')
plt.title('Learning Curve')
plt.legend(loc='best')
plt.show()


In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

models = {
    'Random Forest': rf_model,
    'KNN': knn_model,
    'Logistic Regression': logistic_model,
    'Decision Tree': dt_model,
    'Naive Bayes': nb_model,
    'XGBoost': xgb_model
}

plt.figure(figsize=(10, 8))

for name, model in models.items():
    model.fit(x_train, y_train)  
    y_scores = model.predict_proba(x_test)
    fpr, tpr, _ = roc_curve(y_test, y_scores[:, 1])
    roc_auc = auc(fpr, tpr)  # Ensure auc is the function, not a variable
    if name == 'XGBoost':
        plt.plot(fpr, tpr, label=f'{name} (AUC = {roc_auc:.2f})', color='navy')  
    else:
        plt.plot(fpr, tpr, label=f'{name} (AUC = {roc_auc:.2f})')

plt.plot([0, 1], [0, 1], linestyle='--', color='r', label='Random Guess')

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc="lower right")
plt.show()


In [ ]:
#tester si on a :underfitting /overftting /bestfit
print("Le score de train est: ", round(final_modelRF.score(x_train,y_train),2))
print("Le score de test est: ", round(final_modelRF.score(x_test,y_test),2))

Le score de train est:  1.0
Le score de test est:  0.93


In [ ]:
#Enregistrer le modele dans un fichier
import pickle
with open ("final_modelRF.pkl","wb") as fichier:
    pickle.dump(rf_model ,fichier)


In [ ]:
model = pickle.load(open("final_modelRF.pkl", "rb"))